# Introduction

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

## Instructions

Build a data pipeline using custom functions to extract, transform, aggregate, and load e-commerce data. The SQL query for grocery_sales and the extract() function have already been implemented for you.

To start the project, run the first two cells, then proceed with the following steps:

1. Implement a function named transform() with one argument, taking merged_df as input, filling missing numerical values (using any method of your choice), adding a column "Month", keeping the rows where the weekly sales are over $10,000 and drops the unnecessary columns. Ultimately, it should return a DataFrame and be stored as the clean_data variable.

2. Implement the function avg_weekly_sales_per_month with one argument (the cleaned data). This function will calculate the average monthly sales. For implementing this function you must select the "Month" and "Weekly_Sales" columns as they are the only ones needed for this analysis, then create a chain operation with groupby(), agg(), reset_index(), and round() functions, then group by the "Month" column and calculate the average monthly sales, then call reset_index() to start a new index order and finally round the results to two decimal places.

3. Create a function called load() that takes the cleaned and aggregated DataFrames, and their paths, and saves them as clean_data.csv and agg_data.csv respectively, without an index.

4. Lastly, define a validation() function that checks whether the two csv files from the load() exist in the current working directory.

Note: No engine is required to connect to the database, just execute your query in the SQL code cell provided, and the output will automatically be stored as a pandas DataFrame called grocery_sales, available for you to use in Python code cells. 

In [9]:
# Pre-requisits : Importing libraries and creating source DF

import pandas as pd
import os

grocery_sales = pd.read_csv('store_data.csv', parse_dates=["Date"])
                         
# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

merged_df.head()

,index,Store_ID,Date,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,2010-02-05,1,24924.50,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
1,1,1,2010-02-05,26,11737.12,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
2,2,1,2010-02-05,17,13223.76,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
3,3,1,2010-02-05,45,37.44,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0
4,4,1,2010-02-05,28,1085.29,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0


In [10]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    # 1.- Filling missing numerical values
    df_no_empty = raw_data.fillna(0)
    
    # 2.- Get month column
    df_no_empty['Month'] = raw_data['Date'].dt.month
    
    # 3.- Keeping rows where weekly sales are over 10,000
    df_weekly_sales = df_no_empty.loc[df_no_empty['Weekly_Sales'] > 10000]
    
    # 4.- Dropping unnecessary columns
    df_weekly_sales.drop(['index', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type', 'Size'], inplace = True,                                  axis = 1)
    return df_weekly_sales

In [11]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
clean_data.head()

C:\Users\M9D9V\AppData\Local\Temp\ipykernel_9592\2595918452.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weekly_sales.drop(['index', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type', 'Size'], inplace = True,                                  axis = 1)


,Store_ID,Dept,Weekly_Sales,IsHoliday,CPI,Unemployment,Month
0,1,1,24924.50,0,211.096358,8.106,2.0
1,1,26,11737.12,0,211.096358,8.106,2.0
2,1,17,13223.76,0,211.096358,8.106,2.0
5,1,79,46729.77,0,211.096358,0.000,2.0
6,1,55,21249.31,0,211.096358,0.000,2.0


In [12]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    # 1.- Only selecting Month" and "Weekly_Sales" columns
    df = clean_data[['Month', 'Weekly_Sales']]
    
    # 2.- getting average weekly sales
    df_avg = df.groupby(['Month']).agg("mean").round(2).reset_index()
    df_avg.rename(columns = {'Weekly_Sales' : 'Avg_Sales'}, inplace = True)
    
    return df_avg

In [13]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
agg_data.head()

,Month,Avg_Sales
0,1.0,33174.18
1,2.0,34342.44
2,3.0,33227.31
3,4.0,33414.78
4,5.0,33339.89


In [14]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    # 1.- Exporting full data
    full_data.to_csv(full_data_file_path, index = False)
    
    # 2.- Exporting agg_data
    agg_data.to_csv(agg_data_file_path, index = False)

In [15]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
load(clean_data, 'clean_data.csv', agg_data, 'agg_data.csv')

In [16]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    # 1.- Checking for clean_data.csv
    if os.path.exists('clean_data.csv'):
        print("The file exists.")
    else:
        print("The file does not exist.")
        
    # 2.- Checking for clean_data.csv
    if os.path.exists('clean_data.csv'):
        print("The file exists.")
    else:
        print("The file does not exist.")

In [17]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation(os.getcwd())

The file exists.
The file exists.
